## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-13-08-51-52 +0000,nypost,Historic rains and flooding trigger dramatic r...,https://nypost.com/2025/12/13/us-news/washingt...
1,2025-12-13-08-42-19 +0000,bbc,Witkoff to meet Zelensky for latest Ukraine wa...,https://www.bbc.com/news/articles/c0l9954yr9ko...
2,2025-12-13-08-36-38 +0000,nyt,Thailand Rejects Trump’s Claim That It Reached...,https://www.nytimes.com/2025/12/13/world/asia/...
3,2025-12-13-08-25-51 +0000,nypost,Parkgoer’s death on roller coaster at Universa...,https://nypost.com/2025/12/13/world-news/unive...
4,2025-12-13-08-13-59 +0000,nyt,"Tech Support Scammers Stole $85,000 From Him. ...",https://www.nytimes.com/2025/12/12/business/te...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2364/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
16,trump,45
207,new,15
261,year,11
39,venezuela,10
43,was,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
128,2025-12-12-21-34-00 +0000,wsj,President Trump said he was leaning toward cho...,https://www.wsj.com/economy/central-banking/tr...,121
164,2025-12-12-19-33-00 +0000,wsj,New Epstein Photos Released by Democrats Show ...,https://www.wsj.com/politics/democrats-release...,95
37,2025-12-13-02-16-51 +0000,bbc,"New photos from Epstein estate show Trump, And...",https://www.bbc.com/news/articles/crk777lj763o...,94
226,2025-12-12-15-51-53 +0000,nypost,"Woody Allen, Andrew Mountbatten-Windsor, Bill ...",https://nypost.com/2025/12/12/us-news/woody-al...,93
72,2025-12-13-00-06-15 +0000,nyt,New Photos From Epstein Estate Show Ties to Tr...,https://www.nytimes.com/2025/12/12/us/politics...,92


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
128,121,2025-12-12-21-34-00 +0000,wsj,President Trump said he was leaning toward cho...,https://www.wsj.com/economy/central-banking/tr...
164,50,2025-12-12-19-33-00 +0000,wsj,New Epstein Photos Released by Democrats Show ...,https://www.wsj.com/politics/democrats-release...
314,47,2025-12-12-09-08-17 +0000,nyt,Venezuela’s Nobel Winner Will Go to Norway for...,https://www.nytimes.com/2025/12/06/world/ameri...
30,35,2025-12-13-02-37-44 +0000,nypost,Homeless woman accused of stabbing tourist ins...,https://nypost.com/2025/12/12/us-news/homeless...
54,34,2025-12-13-01-12-49 +0000,nypost,White House hosts ‘Miracle on Ice’ heroes as T...,https://nypost.com/2025/12/12/us-news/hockey-h...
169,34,2025-12-12-19-14-51 +0000,bbc,EU backs indefinite freeze on Russia's frozen ...,https://www.bbc.com/news/articles/c98nnd01g91o...
85,32,2025-12-12-23-35-04 +0000,nypost,Alleged California ‘cult’ leaders arrested for...,https://nypost.com/2025/12/12/us-news/alleged-...
198,32,2025-12-12-17-34-00 +0000,wsj,Amtrak managers are giving up $16 million of t...,https://www.wsj.com/politics/policy/amtrak-wor...
195,31,2025-12-12-17-46-17 +0000,nypost,Disney CEO Bob Iger raises red flags about Net...,https://nypost.com/2025/12/12/business/disney-...
135,30,2025-12-12-21-03-00 +0000,wsj,Reddit Sues Australian Government for Under-16...,https://www.wsj.com/business/media/reddit-sues...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
